In [462]:
## Hidden Markov Model
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn import hmm

SEED = 42
df = pd.read_csv('aggregated.csv')
trend_data = df['nvidia'].values
price_data = df['NVDA Monthly'].values
random.seed(SEED)

In [463]:
# Construct assumption Start Matrix

states = ['Happy', 'Sad', 'Neutral']
start_matrix = np.array([0.5, 0.3, 0.2])

In [464]:
# Constructing the Transition Matrix

# row is t, column is t+1
symbolVocabTrend = {0: "Happy", 1: "Sad", 2: "Neutral"}

# Construct count of transitions from one hidden state to another i.e how many times 0 becomes 1, 1 becomes 2 etc.
transition_counts = {}
for i in range(1, len(trainTrend)):
    transition = (symbolVocabTrend[discrete_trend_data[i-1]], symbolVocabTrend[discrete_trend_data[i]])
    if transition not in transition_counts:
        transition_counts[transition] = 0
    transition_counts[transition] += 1

# print(transition_counts)

# Construct the transition matrix from the counts
transition_matrix = np.zeros((n_hidden, n_hidden))
for i in range(n_hidden):
    for j in range(n_hidden):
        transition = (symbolVocabTrend[i], symbolVocabTrend[j])
        if transition in transition_counts:
            transition_matrix[i, j] = transition_counts[transition] / sum([value for key, value in transition_counts.items() if key[0] == symbolVocabTrend[i]])

# Pretty print the transition matrix with row and column labels
print("Transition Matrix:")
print(pd.DataFrame(transition_matrix, index=["Happy", "Sad", "Neutral"], columns=["Happy", "Sad", "Neutral"]))

### UNIT TESTING ###

# Test the transition matrix
assert np.allclose(transition_matrix.sum(axis=1), np.ones(n_hidden)), "Transition matrix rows should sum to 1"


Transition Matrix:
            Happy       Sad   Neutral
Happy    0.225806  0.387097  0.387097
Sad      0.531250  0.281250  0.187500
Neutral  0.333333  0.523810  0.142857


In [465]:
# Building Price Emission Matrix

# row is emitting column
symbolVocabPrice = {0: "Up", 1: "Down", 2: "Negligible Change"}

# Building Price Emission Matrix

price_symbol_counts = {}

# Iterate over the time series data of trend and price data along with the corresponding hidden states
for trendState, priceAction in zip(trainTrend, trainPrice):
    # Update symbol counts for price data
    emission = (symbolVocabTrend[trendState], symbolVocabPrice[priceAction])
    if emission not in price_symbol_counts:
        price_symbol_counts[emission] = 0
    price_symbol_counts[emission] += 1

# print(price_symbol_counts)

# Construct the emission matrix from the counts
emission_matrix = np.zeros((n_hidden, n_obs))
for i in range(n_hidden):
    for j in range(n_obs):
        emission = (symbolVocabTrend[i], symbolVocabPrice[j])
        if emission in price_symbol_counts:
            emission_matrix[i, j] = price_symbol_counts[emission] / sum([value for key, value in price_symbol_counts.items() if key[0] == symbolVocabTrend[i]])

# Pretty print the emission matrix with row and column labels
print("Emission Matrix:")
print(pd.DataFrame(emission_matrix, index=["Happy", "Sad", "Neutral"], columns=["Up", "Down", "Negligible Change"]))

### UNIT TESTING ###

# Test the emission matrix
assert np.allclose(emission_matrix.sum(axis=1), np.ones(n_hidden)), "Emission matrix rows should sum to 1"

Emission Matrix:
               Up      Down  Negligible Change
Happy    0.580645  0.193548           0.225806
Sad      0.656250  0.250000           0.093750
Neutral  0.636364  0.227273           0.136364


## Hidden Markov Model
Using Viterbi and Forward-Backward to decide states, we draft our plan as such: \



1. q_state = 3 { Happy, Sad, Neutral }

2. obs_states = { Up, Down, Negligible Change }

3. transition matrix is defined as $P(q_{t+1}|q_t)$: \
    Row: Hidden States [$q$], Column: Hidden States [$q$]
4. emission matrix is defined as $P(S_j|q_i)$: \
    Row: Hidden States [$q$], Column: Observable States [$S$]
    

#### Matrix plan v1
Threshold variability


#### Matrix plan v2
Assume hidden states as a distribution

#### Matrix plan v3
Vary backtest period. Last 6 months? Last decade?

#### Notes:
1. Accuracy does not lead to better returns.

#### Housekeeping
I have:
1. Created the transition matrix based on categorising trend data as "Happy, Sad, Neutral".

2. Created the emission matrix based on "Happy emitting Up", etc.

3. Set start matrix as arbitrary.

4. Made a bunch of models to test parameters and versions

5. Run simulation against test price data and keep track of profit as return %

6. Create logger for results

I need to:
1. Do more tests

2. Try VariationalCategoricalHMM

3. Structure document




In [466]:
n_hidden = 3 # states: happy, sad, neutral
n_obs = 3 # observations: up, down, negligible change
traintest_split = 0.7
threshold = 0.02
start_time, stop_time = 110, 232 # length of data: 242

trend_data = trend_data[start_time:stop_time]
price_data = price_data[start_time:stop_time]

## Trend data: 0 - Happy, 1 - Sad, 2 - Neutral
discrete_trend_data = [0 if x >= threshold else 1 if x <= -threshold else 2 for x in trend_data]
## Price data: 0 - Up, 1 - Down, 2 - Negligible change
discrete_price_data = [0 if x >= threshold else 1 if x <= -threshold else 2 for x in price_data]

# Test and Train data
trainTrend, testTrend = discrete_trend_data[:int(len(trend_data)*traintest_split)], discrete_trend_data[int(len(trend_data)*traintest_split):]
trainPrice, discreteTestPrice = discrete_price_data[:int(len(price_data)*traintest_split)], discrete_price_data[int(len(price_data)*traintest_split):]

In [467]:
## Model v1 - Forward-Backward Algorithm

# Initialize the HMM model
modelv1 = hmm.MultinomialHMM(n_components=n_hidden,init_params='',algorithm='map', random_state=SEED)

# Set the model parameters
modelv1.n_features = n_obs
modelv1.startprob_ = start_matrix
modelv1.transmat_ = transition_matrix
modelv1.emissionprob_ = emission_matrix

# Fit the model to the data
X = np.tile(trainPrice, (3, 1)).T
modelv1.fit(X)
logprob, received = modelv1.decode(np.tile(trainPrice, (3, 1)).T)

# Accuracy of most likely sequence of hidden states with respect to the trend data
print("Accuracy: ", sum(1 for received_item, trainTrend_item in zip(received, trainTrend) if received_item == trainTrend_item)/len(received))

# Pretty print the learned transition matrix
print("Learned Transition Matrix:")
print(pd.DataFrame(modelv1.transmat_, index=["Happy", "Sad", "Neutral"], columns=["Happy", "Sad", "Neutral"]))

print("Learned Emission Matrix:")
print(pd.DataFrame(modelv1.emissionprob_, index=["Happy", "Sad", "Neutral"], columns=["Up", "Down", "Negligible Change"]))

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Accuracy:  0.36470588235294116
Learned Transition Matrix:
            Happy       Sad   Neutral
Happy    0.260757  0.354676  0.384567
Sad      0.573801  0.248974  0.177224
Neutral  0.377774  0.481391  0.140835
Learned Emission Matrix:
               Up      Down  Negligible Change
Happy    0.333333  0.333333           0.333333
Sad      0.333333  0.333333           0.333333
Neutral  0.333333  0.333333           0.333333


In [468]:
## Model v2 - Viterbi

# Initialize the HMM model
modelv2 = hmm.MultinomialHMM(n_components=n_hidden,init_params='',algorithm='viterbi', random_state=SEED)

# Set the model parameters
modelv2.n_features = n_obs
modelv2.startprob_ = start_matrix
modelv2.transmat_ = transition_matrix
modelv2.emissionprob_ = emission_matrix

# Fit the model to the data
X = np.tile(trainPrice, (3, 1)).T
modelv2.fit(X)
logprob, received = modelv2.decode(np.tile(trainPrice, (3, 1)).T)

# Accuracy of most likely sequence of hidden states with respect to the trend data
print("Accuracy: ", sum(1 for received_item, trainTrend_item in zip(received, trainTrend) if received_item == trainTrend_item)/len(received))

# Pretty print the learned transition matrix
print("Learned Transition Matrix:")
print(pd.DataFrame(modelv2.transmat_, index=["Happy", "Sad", "Neutral"], columns=["Happy", "Sad", "Neutral"]))

print("Learned Emission Matrix:")
print(pd.DataFrame(modelv2.emissionprob_, index=["Happy", "Sad", "Neutral"], columns=["Up", "Down", "Negligible Change"]))

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Accuracy:  0.2235294117647059
Learned Transition Matrix:
            Happy       Sad   Neutral
Happy    0.260757  0.354676  0.384567
Sad      0.573801  0.248974  0.177224
Neutral  0.377774  0.481391  0.140835
Learned Emission Matrix:
               Up      Down  Negligible Change
Happy    0.333333  0.333333           0.333333
Sad      0.333333  0.333333           0.333333
Neutral  0.333333  0.333333           0.333333


In [469]:
## Model Empty
new_model = hmm.MultinomialHMM(n_components=n_hidden,init_params='ste', random_state=SEED)

new_model.fit(X)
logprob, received = new_model.decode(np.tile(trainPrice, (3, 1)).T)

# Accuracy of most likely sequence of hidden states with respect to the trend data
print("Accuracy: ", sum(1 for received_item, trainTrend_item in zip(received, trainTrend) if received_item == trainTrend_item)/len(received))

# Pretty print the learned transition matrix
print("Learned Transition Matrix:")
print(pd.DataFrame(new_model.transmat_, index=["Happy", "Sad", "Neutral"], columns=["Happy", "Sad", "Neutral"]))


print("Learned Emission Matrix:")
print(pd.DataFrame(new_model.emissionprob_, index=["Happy", "Sad", "Neutral"], columns=["Up", "Down", "Negligible Change"]))

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Accuracy:  0.25882352941176473
Learned Transition Matrix:
            Happy       Sad   Neutral
Happy    0.001151  0.998823  0.000026
Sad      0.968256  0.007915  0.023830
Neutral  0.497101  0.138508  0.364391
Learned Emission Matrix:
               Up      Down  Negligible Change
Happy    0.333333  0.333333           0.333333
Sad      0.333333  0.333333           0.333333
Neutral  0.333333  0.333333           0.333333


In [ ]:
## Model v3 - Variational Inference

# Initialize the HMM model
modelv3 = hmm.VariationalCategoricalHMM(n_components=n_hidden,init_params='',algorithm='map', random_state=SEED)

# Set the model parameters
modelv3.n_features = n_obs
modelv3.startprob_ = start_matrix
modelv3.transmat_ = transition_matrix
modelv3.emissionprob_ = emission_matrix

# Fit the model to the data
X = np.tile(trainPrice, (3, 1)).T
modelv3.fit(X)
logprob, received = modelv3.decode(np.tile(trainPrice, (3, 1)).T)

# Accuracy of most likely sequence of hidden states with respect to the trend data
print("Accuracy: ", sum(1 for received_item, trainTrend_item in zip(received, trainTrend) if received_item == trainTrend_item)/len(received))

# Pretty print the learned transition matrix
print("Learned Transition Matrix:")
print(pd.DataFrame(modelv3.transmat_, index=["Happy", "Sad", "Neutral"], columns=["Happy", "Sad", "Neutral"]))

print("Learned Emission Matrix:")
print(pd.DataFrame(modelv3.emissionprob_, index=["Happy", "Sad", "Neutral"], columns=["Up", "Down", "Negligible Change"]))

In [470]:
# Predict the hidden states of the test data
X_test = np.tile(discreteTestPrice, (3, 1)).T

logprob, modelv1received = modelv1.decode(X_test)
logprob, modelv2received = modelv2.decode(X_test)
logprob, nullmodelreceived = new_model.decode(X_test)

test = pd.read_csv('OpenClosePrice.csv')
testPrice = (test['Close'].values+test['Open'].values)/2
testPrice = testPrice[start_time:stop_time]
testPrice = testPrice[int(len(testPrice)*traintest_split):]

# Buy or Sell or Hold based on the hidden states, calculate PnL based on the test price data
currA = [1000, 0] # cash in hand, position
currB = [1000, 0]
currC = [1000, 0]
currBase = [1000, 0]

for i in range(len(testPrice)):
    if i == 0:
        currBase[0] -= testPrice[i]
        currBase[1] += 1

    # Model v1
    if modelv1received[i] == 0:
        currA[0] -= testPrice[i]
        currA[1] += 1
    elif modelv1received[i] == 1:
        currA[0] += testPrice[i]
        currA[1] -= 1

    # Model v2
    if modelv2received[i] == 0:
        currB[0] -= testPrice[i]
        currB[1] += 1
    elif modelv2received[i] == 1:
        currB[0] += testPrice[i]
        currB[1] -= 1

    # Null Model
    if nullmodelreceived[i] == 0:
        currC[0] -= testPrice[i]
        currC[1] += 1
    elif nullmodelreceived[i] == 1:
        currC[0] += testPrice[i]
        currC[1] -= 1

    if i == len(testPrice): # Exit position at the end
        currA[0] += currA[1]*testPrice[i]
        currA[1] = 0
        currB[0] += currB[1]*testPrice[i]
        currB[1] = 0
        currC[0] += currC[1]*testPrice[i]
        currC[1] = 0
        currBase[0] += currBase[1]*testPrice[i]
        currBase[1] = 0

print("Model v1 - Forward Backward Return on Investment: ", (currA[0]-1000)/10)
print("Model v2 - Viterbi Return on Investment: ", (currB[0]-1000)/10)
print("Null Model Return on Investment: ", (currC[0]-1000)/10)
print("Base Model Return on Investment: ", (currBase[0]-1000)/10)







Model v1 - Forward Backward Return on Investment:  -650.7733773362582
Model v2 - Viterbi Return on Investment:  -6.950748872047598
Null Model Return on Investment:  18.085689243641173
Base Model Return on Investment:  -6.826545745829094


In [1]:
## Logger

with open('logs.csv', 'w') as f:
    f.write("Model,Parameters,Return on Investment,\n") # Parameters contain backtest period, threshold
